In [1]:
from evoVAE.utils.datasets import MSA_Dataset
import evoVAE.utils.seq_tools as st
from evoVAE.models.seqVAE import SeqVAE
from evoVAE.trainer.seqVAE_train import seq_train
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import wandb

#### Config

In [2]:
wandb.init(
    project="SeqVAE_training",

    # hyperparameters
    config = {

        # Dataset info
        "dataset": "PhoQ",
        "seq_theta": 0.2, # reweighting 
        "AA_count": 21, # standard AA + gap
        
        # ADAM 
        "learning_rate": 1e-5, # ADAM
        "weight_decay": 0.01, # ADAM

        # Hidden units 
        "momentum": 0.9, 
        "dropout": 0.5,

        # Training loop 
        "epochs": 1,
        "batch_size": 2,
        "max_norm": 1.0, # gradient clipping
        
        # Model info
        "architecture": "SeqVAE",
        "latent_dims": 2,
        "hidden_dims": [32, 16],
    }
)

config = wandb.config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sebastian-porras01. Use `wandb login --relogin` to force relogin


#### Data loading and preprocessing

In [3]:

# Read in the datasets and create train and validation sets 
alns: pd.DataFrame = st.read_aln_file("../data/alignments/tiny.aln")
train, val = train_test_split(alns, test_size=0.2)

# create one-hot encodings and calculate reweightings 

# TRAINING 
train_encodings, train_weights = st.encode_and_weight_seqs(train["sequence"],theta=config.seq_theta)
train_ids = train["id"].values # just the seq identifiers 
train_dataset = MSA_Dataset(train_encodings, train_weights, train_ids)

# VALIDATION
val_encodings, val_weights = st.encode_and_weight_seqs(val["sequence"], theta=config.seq_theta)
val_ids = val["id"].values
val_dataset = MSA_Dataset(val_encodings, val_weights, val_ids)


# DATA LOADERS #
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
#next(iter(train_loader))[0].shape,next(iter(train_loader))[1].shape, next(iter(train_loader))[2]

Encoding the sequences and calculating weights
The sequence encoding tensor has size: torch.Size([2, 5, 21])
The sequence weight array has size: (2,)

Encoding the sequences and calculating weights
The sequence encoding tensor has size: torch.Size([1, 5, 21])
The sequence weight array has size: (1,)



In [ ]:
#encoding, weights, id = train_dataset[0]
#print(encoding.shape, weights, id)

# translation = st.one_hot_to_seq(encoding)
# print(translation)

#### Create the model

In [5]:
# get the sequence length 
seq_len = train_dataset[0][0].shape[0]
input_dims = seq_len * config.AA_count

# use preset structure for hidden dimensions 
model = SeqVAE(input_dims=input_dims, latent_dims=config.latent_dims, hidden_dims=config.hidden_dims, config=config) 
# model

SeqVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=105, out_features=16, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Dropout(p=0.5, inplace=False)
      (3): Linear(in_features=16, out_features=16, bias=True)
      (4): LeakyReLU(negative_slope=0.01)
      (5): BatchNorm1d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Dropout(p=0.5, inplace=False)
      (3): Linear(in_features=32, out_features=32, bias=True)
      (4): LeakyReLU(negative_slope=0.01)
      (5): BatchNorm1d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    )
  )
  (z_mu_sampler): Linear(in_features=32, out_features=2, bias=True)
  (z_logvar_sampler): Linear(in_features=32, out_features=2, bias=True)
  (upscale_z): Linear(in_features=2, out_features=32, bias=True)
  (decoder): Sequential

#### Training Loop

In [6]:
trained_model = seq_train(model, train_loader=train_loader, val_loader=val_loader, device=device, config=config)

/Users/sebs_mac/miniconda3/envs/embed/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
wandb.finish()

epoch_ELBO,▁
epoch_Gauss_likelihood,▁
epoch_KLD,▁
epoch_val_ELBO,▁
epoch_val_Gauss_likelihood,▁
epoch_val_KLD,▁
epoch_ELBO,654.53662
epoch_Gauss_likelihood,-654.61841
epoch_KLD,-0.0818
epoch_val_ELBO,607.16467
epoch_val_Gauss_likelihood,-607.3479
